In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline
plt.style.use('ggplot')

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from keras import metrics

In [3]:
data = pd.read_csv("data/all.csv")
data

,x,y,z,t
0,0.0,192.0,0.0,idle
1,0.0,-64.0,0.0,idle
2,64.0,64.0,0.0,idle
3,0.0,0.0,0.0,idle
4,0.0,-192.0,-64.0,idle
...,...,...,...,...
26675,960.0,-10944.0,-3904.0,run
26676,3456.0,5952.0,512.0,run
26677,2880.0,6656.0,832.0,run
26678,960.0,-12160.0,-704.0,run


In [4]:
window_size = 25
step_size = 10

In [5]:
def segment_signal(data, window_size = window_size, step_size = step_size):
    segments = np.empty((0, window_size, 3))
    labels = np.empty((0))

    for activity in data["t"].unique():
        df = data[data['t']==activity]
        
        for i in range(0, len(df)-window_size, step_size):
            start = i
            end = i + window_size
            x = df["x"][start:end]
            y = df["y"][start:end]
            z = df["z"][start:end]
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels, stats.mode(df["t"][start:end])[0][0])

    return segments, labels

# segment data into windows
segments, labels = segment_signal(data)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
segments.shape

(2662, 25, 3)

In [6]:
# train-test split
train_test_split = np.random.rand(len(segments)) < 0.80
train_x = segments[train_test_split]
train_y = labels[train_test_split]
test_x = segments[~train_test_split]
test_y = labels[~train_test_split]

In [10]:
time_step = window_size
num_channels = 3
num_labels = 3

epochs = 3000
batch_size = segments.shape[0]

In [16]:
# construct CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, input_shape=(time_step, num_channels), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Flatten())
model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 23, 64)            640       
_________________________________________________________________
dropout_5 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 23, 100)           6500      
_________________________________________________________________
dropout_6 (Dropout)          (None, 23, 100)           0         
_________________________________________________________________
dense_9 (Dense)              (None, 23, 100)           10100     
_________________________________________________________________
flatten_3 (Flatten)          (None, 2300)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                

In [17]:
# compile model
model.compile(Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# set up checkpoints
filepath="model-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [13]:
history = model.fit(train_x, train_y, validation_split=0.1, batch_size=batch_size, epochs=epochs, callbacks=callbacks, shuffle=True, verbose=2)


Train on 1927 samples, validate on 215 samples
Epoch 1/3000
 - 1s - loss: 543.7745 - accuracy: 0.3191 - val_loss: 1912.6975 - val_accuracy: 0.0000e+00
Epoch 2/3000


/home/vernon/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


 - 0s - loss: 837.1723 - accuracy: 0.3596 - val_loss: 1126.8696 - val_accuracy: 0.0372
Epoch 3/3000
 - 0s - loss: 526.0139 - accuracy: 0.3461 - val_loss: 260.5857 - val_accuracy: 0.4233
Epoch 4/3000
 - 0s - loss: 455.1602 - accuracy: 0.3114 - val_loss: 33.7068 - val_accuracy: 0.8837
Epoch 5/3000
 - 0s - loss: 564.1318 - accuracy: 0.2963 - val_loss: 26.0088 - val_accuracy: 0.9116
Epoch 6/3000
 - 0s - loss: 548.4963 - accuracy: 0.3025 - val_loss: 77.0991 - val_accuracy: 0.7070
Epoch 7/3000
 - 0s - loss: 388.5023 - accuracy: 0.3316 - val_loss: 254.3476 - val_accuracy: 0.3256
Epoch 8/3000
 - 0s - loss: 359.5038 - accuracy: 0.3565 - val_loss: 429.9621 - val_accuracy: 0.1535
Epoch 9/3000
 - 0s - loss: 382.7830 - accuracy: 0.3716 - val_loss: 442.1947 - val_accuracy: 0.1302
Epoch 10/3000
 - 0s - loss: 412.0547 - accuracy: 0.3700 - val_loss: 314.0079 - val_accuracy: 0.2140
Epoch 11/3000
 - 0s - loss: 335.1476 - accuracy: 0.3736 - val_loss: 145.0493 - val_accuracy: 0.4558
Epoch 12/3000
 - 0s - l

Epoch 86/3000
 - 0s - loss: 59.5278 - accuracy: 0.4546 - val_loss: 1.3041 - val_accuracy: 0.9395
Epoch 87/3000
 - 0s - loss: 54.5005 - accuracy: 0.4395 - val_loss: 1.2922 - val_accuracy: 0.9395
Epoch 88/3000
 - 0s - loss: 54.0424 - accuracy: 0.4504 - val_loss: 1.2989 - val_accuracy: 0.9349
Epoch 89/3000
 - 0s - loss: 53.0713 - accuracy: 0.4515 - val_loss: 1.2517 - val_accuracy: 0.9349
Epoch 90/3000
 - 0s - loss: 53.7239 - accuracy: 0.4338 - val_loss: 1.1525 - val_accuracy: 0.9349
Epoch 91/3000
 - 0s - loss: 50.0304 - accuracy: 0.4421 - val_loss: 1.1107 - val_accuracy: 0.9442
Epoch 92/3000
 - 0s - loss: 54.0137 - accuracy: 0.4395 - val_loss: 1.0629 - val_accuracy: 0.9442
Epoch 93/3000
 - 0s - loss: 50.8703 - accuracy: 0.4520 - val_loss: 0.9634 - val_accuracy: 0.9488
Epoch 94/3000
 - 0s - loss: 55.1606 - accuracy: 0.4338 - val_loss: 0.8980 - val_accuracy: 0.9488
Epoch 95/3000
 - 0s - loss: 48.1063 - accuracy: 0.4504 - val_loss: 0.8594 - val_accuracy: 0.9442
Epoch 96/3000
 - 0s - loss: 44

Epoch 170/3000
 - 0s - loss: 18.6445 - accuracy: 0.5143 - val_loss: 0.0375 - val_accuracy: 0.9953
Epoch 171/3000
 - 0s - loss: 19.0062 - accuracy: 0.5029 - val_loss: 0.0451 - val_accuracy: 0.9953
Epoch 172/3000
 - 0s - loss: 19.0430 - accuracy: 0.5054 - val_loss: 0.0501 - val_accuracy: 0.9907
Epoch 173/3000
 - 0s - loss: 19.4146 - accuracy: 0.5179 - val_loss: 0.0491 - val_accuracy: 0.9953
Epoch 174/3000
 - 0s - loss: 21.3742 - accuracy: 0.4992 - val_loss: 0.0419 - val_accuracy: 0.9953
Epoch 175/3000
 - 0s - loss: 18.0614 - accuracy: 0.4997 - val_loss: 0.0334 - val_accuracy: 0.9953
Epoch 176/3000
 - 0s - loss: 18.0684 - accuracy: 0.4966 - val_loss: 0.0261 - val_accuracy: 0.9953
Epoch 177/3000
 - 0s - loss: 18.2441 - accuracy: 0.5044 - val_loss: 0.0186 - val_accuracy: 0.9953
Epoch 178/3000
 - 0s - loss: 17.1459 - accuracy: 0.5205 - val_loss: 0.0132 - val_accuracy: 0.9953
Epoch 179/3000
 - 0s - loss: 16.1372 - accuracy: 0.5200 - val_loss: 0.0092 - val_accuracy: 0.9953
Epoch 180/3000
 - 0s

...

Epoch 2590/3000
 - 0s - loss: 0.1374 - accuracy: 0.9569 - val_loss: 0.0572 - val_accuracy: 0.9767
Epoch 2591/3000
 - 0s - loss: 0.1153 - accuracy: 0.9595 - val_loss: 0.0546 - val_accuracy: 0.9814
Epoch 2592/3000
 - 0s - loss: 0.1320 - accuracy: 0.9538 - val_loss: 0.0487 - val_accuracy: 0.9814
Epoch 2593/3000
 - 0s - loss: 0.1309 - accuracy: 0.9595 - val_loss: 0.0448 - val_accuracy: 0.9814
Epoch 2594/3000
 - 0s - loss: 0.1390 - accuracy: 0.9538 - val_loss: 0.0452 - val_accuracy: 0.9814
Epoch 2595/3000
 - 0s - loss: 0.1571 - accuracy: 0.9538 - val_loss: 0.0460 - val_accuracy: 0.9814
Epoch 2596/3000
 - 0s - loss: 0.1307 - accuracy: 0.9595 - val_loss: 0.0480 - val_accuracy: 0.9814
Epoch 2597/3000
 - 0s - loss: 0.1228 - accuracy: 0.9616 - val_loss: 0.0480 - val_accuracy: 0.9814
Epoch 2598/3000
 - 0s - loss: 0.1839 - accuracy: 0.9528 - val_loss: 0.0473 - val_accuracy: 0.9767
Epoch 2599/3000
 - 0s - loss: 0.1502 - accuracy: 0.9543 - val_loss: 0.0451 - val_accuracy: 0.9814
Epoch 2600/3000
 - 0

Epoch 2674/3000
 - 0s - loss: 0.1400 - accuracy: 0.9559 - val_loss: 0.0473 - val_accuracy: 0.9767
Epoch 2675/3000
 - 0s - loss: 0.1330 - accuracy: 0.9626 - val_loss: 0.0511 - val_accuracy: 0.9767
Epoch 2676/3000
 - 0s - loss: 0.1640 - accuracy: 0.9497 - val_loss: 0.0543 - val_accuracy: 0.9767
Epoch 2677/3000
 - 0s - loss: 0.1661 - accuracy: 0.9486 - val_loss: 0.0570 - val_accuracy: 0.9767
Epoch 2678/3000
 - 0s - loss: 0.1461 - accuracy: 0.9549 - val_loss: 0.0539 - val_accuracy: 0.9767
Epoch 2679/3000
 - 0s - loss: 0.1145 - accuracy: 0.9626 - val_loss: 0.0485 - val_accuracy: 0.9814
Epoch 2680/3000
 - 0s - loss: 0.1504 - accuracy: 0.9574 - val_loss: 0.0433 - val_accuracy: 0.9814
Epoch 2681/3000
 - 0s - loss: 0.1289 - accuracy: 0.9616 - val_loss: 0.0400 - val_accuracy: 0.9814
Epoch 2682/3000
 - 0s - loss: 0.1241 - accuracy: 0.9569 - val_loss: 0.0380 - val_accuracy: 0.9814
Epoch 2683/3000
 - 0s - loss: 0.1378 - accuracy: 0.9543 - val_loss: 0.0382 - val_accuracy: 0.9814
Epoch 2684/3000
 - 0

Epoch 2758/3000
 - 0s - loss: 0.1354 - accuracy: 0.9507 - val_loss: 0.0504 - val_accuracy: 0.9814
Epoch 2759/3000
 - 0s - loss: 0.1388 - accuracy: 0.9491 - val_loss: 0.0462 - val_accuracy: 0.9814
Epoch 2760/3000
 - 0s - loss: 0.1318 - accuracy: 0.9590 - val_loss: 0.0423 - val_accuracy: 0.9814
Epoch 2761/3000
 - 0s - loss: 0.1412 - accuracy: 0.9538 - val_loss: 0.0381 - val_accuracy: 0.9814
Epoch 2762/3000
 - 0s - loss: 0.1352 - accuracy: 0.9538 - val_loss: 0.0330 - val_accuracy: 0.9860
Epoch 2763/3000
 - 0s - loss: 0.1241 - accuracy: 0.9606 - val_loss: 0.0303 - val_accuracy: 0.9860
Epoch 2764/3000
 - 0s - loss: 0.1300 - accuracy: 0.9491 - val_loss: 0.0313 - val_accuracy: 0.9860
Epoch 2765/3000
 - 0s - loss: 0.1325 - accuracy: 0.9606 - val_loss: 0.0321 - val_accuracy: 0.9860
Epoch 2766/3000
 - 0s - loss: 0.1254 - accuracy: 0.9580 - val_loss: 0.0346 - val_accuracy: 0.9860
Epoch 2767/3000
 - 0s - loss: 0.1279 - accuracy: 0.9580 - val_loss: 0.0425 - val_accuracy: 0.9814
Epoch 2768/3000
 - 0

Epoch 2842/3000
 - 0s - loss: 0.1225 - accuracy: 0.9585 - val_loss: 0.0503 - val_accuracy: 0.9814
Epoch 2843/3000
 - 0s - loss: 0.1301 - accuracy: 0.9569 - val_loss: 0.0488 - val_accuracy: 0.9814
Epoch 2844/3000
 - 0s - loss: 0.1455 - accuracy: 0.9538 - val_loss: 0.0492 - val_accuracy: 0.9814
Epoch 2845/3000
 - 0s - loss: 0.1379 - accuracy: 0.9590 - val_loss: 0.0519 - val_accuracy: 0.9814
Epoch 2846/3000
 - 0s - loss: 0.1103 - accuracy: 0.9621 - val_loss: 0.0532 - val_accuracy: 0.9814
Epoch 2847/3000
 - 0s - loss: 0.1146 - accuracy: 0.9626 - val_loss: 0.0496 - val_accuracy: 0.9814
Epoch 2848/3000
 - 0s - loss: 0.1381 - accuracy: 0.9590 - val_loss: 0.0480 - val_accuracy: 0.9814
Epoch 2849/3000
 - 0s - loss: 0.1433 - accuracy: 0.9528 - val_loss: 0.0486 - val_accuracy: 0.9814
Epoch 2850/3000
 - 0s - loss: 0.1417 - accuracy: 0.9507 - val_loss: 0.0496 - val_accuracy: 0.9814
Epoch 2851/3000
 - 0s - loss: 0.1103 - accuracy: 0.9611 - val_loss: 0.0504 - val_accuracy: 0.9814
Epoch 2852/3000
 - 0

Epoch 2926/3000
 - 0s - loss: 0.1291 - accuracy: 0.9554 - val_loss: 0.0487 - val_accuracy: 0.9814
Epoch 2927/3000
 - 0s - loss: 0.1228 - accuracy: 0.9564 - val_loss: 0.0432 - val_accuracy: 0.9814
Epoch 2928/3000
 - 0s - loss: 0.1171 - accuracy: 0.9580 - val_loss: 0.0427 - val_accuracy: 0.9814
Epoch 2929/3000
 - 0s - loss: 0.1126 - accuracy: 0.9580 - val_loss: 0.0436 - val_accuracy: 0.9814
Epoch 2930/3000
 - 0s - loss: 0.1150 - accuracy: 0.9595 - val_loss: 0.0468 - val_accuracy: 0.9814
Epoch 2931/3000
 - 0s - loss: 0.1250 - accuracy: 0.9590 - val_loss: 0.0495 - val_accuracy: 0.9767
Epoch 2932/3000
 - 0s - loss: 0.1021 - accuracy: 0.9663 - val_loss: 0.0529 - val_accuracy: 0.9767
Epoch 2933/3000
 - 0s - loss: 0.1316 - accuracy: 0.9569 - val_loss: 0.0549 - val_accuracy: 0.9721
Epoch 2934/3000
 - 0s - loss: 0.1361 - accuracy: 0.9564 - val_loss: 0.0570 - val_accuracy: 0.9721
Epoch 2935/3000
 - 0s - loss: 0.1138 - accuracy: 0.9626 - val_loss: 0.0601 - val_accuracy: 0.9721
Epoch 2936/3000
 - 0

In [14]:
_, accuracy = model.evaluate(test_x, test_y, batch_size=test_x.shape[0], verbose=2)
accuracy

0.9903846383094788

In [15]:
model.save('har_model.hdf5')